## Data pre-processing and extractor for US Shipments


Code below was used to to create parquet files and extract the data we needed for our EDA / Descriptive analytics. The strategy was to write write the files into our S3 buckets and transfer it between the members for quick access with the files. All queries below after fetching the data will write it in the S3 bucket as a csv file.

In [46]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
import boto3
import pyspark.sql.functions as F
from pyspark.sql.functions import count
import pandas as pd
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
dir_ = 's3-bucket-bdcc2023-lab2'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### S3 Object Initialize

In [48]:
# Initialize S3 Access
aws_access_key_id = ''
aws_secret_access_key = ''

session = boto3.Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
s3 = session.client('s3')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create Parquet Files

In [49]:
# Create parquet for header table
# df_h1 = spark.read.csv(f's3://{dir_}/ams__header_2018__202001290000.csv', header=True)
# df_h2 = spark.read.csv(f's3://{dir_}/ams__header_2019__202001080000.csv', header=True)
# df_h3 = spark.read.csv(f's3://{dir_}/ams__header_2020__202009291500.csv', header=True)

# union_df = df_h1.union(df_h2)
# union_df = union_df.union(df_h3)

# union_df.write.mode('overwrite').parquet(f's3://{dir_}/header.parquet')
# df_h1, df_h2, df_h3 = None, None, None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
# Create parquet for container table
# df_c1 = spark.read.csv(f's3://{dir_}/ams__container_2018__202001290000.csv', header=True)
# df_c2 = spark.read.csv(f's3://{dir_}/ams__container_2019__202001080000.csv', header=True)
# df_c3 = spark.read.csv(f's3://{dir_}/ams__container_2020__202009291500.csv', header=True)

# union_df = df_c1.union(df_c2)
# union_df = union_df.union(df_c3)

# union_df.write.mode('overwrite').parquet(f's3://{dir_}/container.parquet')
# df_c1, df_c2, df_c3 = None, None, None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
# Create parquet for container table
# df_s1 = spark.read.csv(f's3://{dir_}/ams__shipper_2018__202001290000.csv', header=True)
# df_s2 = spark.read.csv(f's3://{dir_}/ams__shipper_2019__202001080000.csv', header=True)
# df_s3 = spark.read.csv(f's3://{dir_}/ams__shipper_2020__202009291500.csv', header=True)

# union_df = df_s1.union(df_s2)
# union_df = union_df.union(df_s3)

# union_df.write.mode('overwrite').parquet(f's3://{dir_}/shipper.parquet')
# df_s1, df_s2, df_s3 = None, None, None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
# Country Name Dictionary Mapping

country_codes = {
    'AF': 'Afghanistan',
    'AL': 'Albania',
    'DZ': 'Algeria',
    'AD': 'Andorra',
    'AO': 'Angola',
    'AI': 'Anguilla',
    'AG': 'Antigua and Barbuda',
    'AR': 'Argentina',
    'AM': 'Armenia',
    'AW': 'Aruba',
    'AU': 'Australia',
    'AT': 'Austria',
    'AZ': 'Azerbaijan',
    'BS': 'Bahamas',
    'BH': 'Bahrain',
    'BD': 'Bangladesh',
    'BB': 'Barbados',
    'BY': 'Belarus',
    'BE': 'Belgium',
    'BZ': 'Belize',
    'BJ': 'Benin',
    'BM': 'Bermuda',
    'BT': 'Bhutan',
    'BO': 'Bolivia',
    'BA': 'Bosnia and Herzegovina',
    'BW': 'Botswana',
    'BR': 'Brazil',
    'VG': 'British Virgin Islands',
    'BN': 'Brunei',
    'BG': 'Bulgaria',
    'BF': 'Burkina Faso',
    'BI': 'Burundi',
    'KH': 'Cambodia',
    'CM': 'Cameroon',
    'CA': 'Canada',
    'CV': 'Cape Verde',
    'KY': 'Cayman Islands',
    'CF': 'Central African Republic',
    'TD': 'Chad',
    'CL': 'Chile',
    'CN': 'China',
    'CO': 'Colombia',
    'KM': 'Comoros',
    'CG': 'Congo',
    'CK': 'Cook Islands',
    'CR': 'Costa Rica',
    'HR': 'Croatia',
    'CU': 'Cuba',
    'CY': 'Cyprus',
    'CZ': 'Czech Republic',
    'DK': 'Denmark',
    'DJ': 'Djibouti',
    'DM': 'Dominica',
    'DO': 'Dominican Republic',
    'TL': 'East Timor',
    'EC': 'Ecuador',
    'EG': 'Egypt',
    'SV': 'El Salvador',
    'GQ': 'Equatorial Guinea',
    'ER': 'Eritrea',
    'EE': 'Estonia',
    'ET': 'Ethiopia',
    'FK': 'Falkland Islands',
    'FO': 'Faroe Islands',
    'FJ': 'Fiji',
    'FI': 'Finland',
    'FR': 'France',
    'GF': 'French Guiana',
    'PF': 'French Polynesia',
    'GA': 'Gabon',
    'GM': 'Gambia',
    'GE': 'Georgia',
    'DE': 'Germany',
    'GH': 'Ghana',
    'GI': 'Gibraltar',
    'GR': 'Greece',
    'GL': 'Greenland',
    'GD': 'Grenada',
    'GP': 'Guadeloupe',
    'GT': 'Guatemala',
    'GN': 'Guinea',
    'GW': 'Guinea-Bissau',
    'GY': 'Guyana',
    'HT': 'Haiti',
    'HN': 'Honduras',
    'HK': 'Hong Kong',
    'HU': 'Hungary',
    'IS': 'Iceland',
    'IN': 'India',
    'ID': 'Indonesia',
    'IR': 'Iran',
    'IQ': 'Iraq',
    'IE': 'Ireland',
    'IL': 'Israel',
    'IT': 'Italy',
    'JM': 'Jamaica',
    'JP': 'Japan',
    'JO': 'Jordan',
    'KZ': 'Kazakhstan',
    'KE': 'Kenya',
    'KI': 'Kiribati',
    'KW': 'Kuwait',
    'KG': 'Kyrgyzstan',
    'LA': 'Laos',
    'LV': 'Latvia',
    'LB': 'Lebanon',
    'LS': 'Lesotho',
    'LR': 'Liberia',
    'LY': 'Libya',
    'LI': 'Liechtenstein',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'MO': 'Macau',
    'MK': 'North Macedonia',
    'MG': 'Madagascar',
    'MW': 'Malawi',
    'MY': 'Malaysia',
    'MV': 'Maldives',
    'ML': 'Mali',
    'MT': 'Malta',
    'MH': 'Marshall Islands',
    'MQ': 'Martinique',
    'MR': 'Mauritania',
    'MU': 'Mauritius',
    'YT': 'Mayotte',
    'MX': 'Mexico',
    'FM': 'Micronesia',
    'MD': 'Moldova',
    'MC': 'Monaco',
    'MN': 'Mongolia',
    'ME': 'Montenegro',
    'MS': 'Montserrat',
    'MA': 'Morocco',
    'MZ': 'Mozambique',
    'MM': 'Myanmar',
    'NA': 'Namibia',
    'NR': 'Nauru',
    'NP': 'Nepal',
    'NL': 'Netherlands',
    'NC': 'New Caledonia',
    'NZ': 'New Zealand',
    'NI': 'Nicaragua',
    'NE': 'Niger',
    'NG': 'Nigeria',
    'NU': 'Niue',
    'NF': 'Norfolk Island',
    'KP': 'North Korea',
    'NO': 'Norway',
    'OM': 'Oman',
    'PK': 'Pakistan',
    'PW': 'Palau',
    'PA': 'Panama',
    'PG': 'Papua New Guinea',
    'PY': 'Paraguay',
    'PE': 'Peru',
    'PH': 'Philippines',
    'PL': 'Poland',
    'PT': 'Portugal',
    'QA': 'Qatar',
    'RE': 'Réunion',
    'RO': 'Romania',
    'RU': 'Russia',
    'RW': 'Rwanda',
    'BL': 'Saint Barthélemy',
    'SH': 'Saint Helena',
    'KN': 'Saint Kitts and Nevis',
    'LC': 'Saint Lucia',
    'MF': 'Saint Martin',
    'PM': 'Saint Pierre and Miquelon',
    'VC': 'Saint Vincent and the Grenadines',
    'WS': 'Samoa',
    'SM': 'San Marino',
    'ST': 'São Tomé and Príncipe',
    'SA': 'Saudi Arabia',
    'SN': 'Senegal',
    'RS': 'Serbia',
    'SC': 'Seychelles',
    'SL': 'Sierra Leone',
    'SG': 'Singapore',
    'SK': 'Slovakia',
    'SI': 'Slovenia',
    'SB': 'Solomon Islands',
    'SO': 'Somalia',
    'ZA': 'South Africa',
    'KR': 'South Korea',
    'SS': 'South Sudan',
    'ES': 'Spain',
    'LK': 'Sri Lanka',
    'SD': 'Sudan',
    'SR': 'Suriname',
    'SZ': 'Eswatini',
    'SE': 'Sweden',
    'CH': 'Switzerland',
    'SY': 'Syria',
    'TW': 'Taiwan',
    'TJ': 'Tajikistan',
    'TZ': 'Tanzania',
    'TH': 'Thailand',
    'TG': 'Togo',
    'TO': 'Tonga',
    'TT': 'Trinidad and Tobago',
    'TN': 'Tunisia',
    'TR': 'Turkey',
    'TM': 'Turkmenistan',
    'TC': 'Turks and Caicos Islands',
    'TV': 'Tuvalu',
    'UG': 'Uganda',
    'UA': 'Ukraine',
    'AE': 'United Arab Emirates',
    'GB': 'United Kingdom',
    'US': 'United States',
    'UY': 'Uruguay',
    'UZ': 'Uzbekistan',
    'VU': 'Vanuatu',
    'VA': 'Vatican City',
    'VE': 'Venezuela',
    'VN': 'Vietnam',
    'WF': 'Wallis and Futuna',
    'YE': 'Yemen',
    'ZM': 'Zambia',
    'ZW': 'Zimbabwe'
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Feature Engineering and Loading of Parquet Files

In [53]:
df_header = (spark.read.parquet(f's3://{dir_}/header.parquet')
                  .where((F.col('vessel_country_code').isNotNull()) & 
                         (F.col('actual_arrival_date').isNotNull()) & 
                         (F.col('estimated_arrival_date') >= '2018-01-01') &
                         (F.col('actual_arrival_date') >= '2018-01-01') &
                         (F.col('estimated_arrival_date') <= '2020-12-31') &
                         (F.col('actual_arrival_date') <= '2020-12-31') 
                         )
                  .withColumn('actual_year', F.year('actual_arrival_date'))
                  .withColumn('actual_month', F.month('actual_arrival_date'))
                  .withColumn('estimated_year', F.year('estimated_arrival_date'))
                  .withColumn('estimated_month', F.month('estimated_arrival_date'))
                  .withColumn('actual_year_month', F.date_format('actual_arrival_date', 'yyyy-MM'))
                  .withColumn('estimated_year_month', F.date_format('estimated_arrival_date', 'yyyy-MM'))
                  .withColumn('delay', F.datediff(F.col('actual_arrival_date'), 
                                                     F.col('estimated_arrival_date')))
                     .withColumn('is_delayed', F.when(F.col("delay") > 0, 'delayed')
                                               .when(F.col("delay") < 0, 'early')
                                               .otherwise('on schedule')).cache()
                 )
df_container = spark.read.parquet(f's3://{dir_}/container.parquet') 
df_shipper = spark.read.parquet(f's3://{dir_}/shipper.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [264]:
print('Header Row Count: ' + str(df_header.count()))
print('Container Row Count: ' + str(df_container.count()))
print('Shipper Row Count: ' + str(df_shipper.count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Header Row Count: 53844894
Container Row Count: 85503338
Shipper Row Count: 40240366

In [265]:
df_header.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- identifier: string (nullable = true)
 |-- carrier_code: string (nullable = true)
 |-- vessel_country_code: string (nullable = true)
 |-- vessel_name: string (nullable = true)
 |-- port_of_unlading: string (nullable = true)
 |-- estimated_arrival_date: string (nullable = true)
 |-- foreign_port_of_lading_qualifier: string (nullable = true)
 |-- foreign_port_of_lading: string (nullable = true)
 |-- manifest_quantity: string (nullable = true)
 |-- manifest_unit: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- weight_unit: string (nullable = true)
 |-- measurement: string (nullable = true)
 |-- measurement_unit: string (nullable = true)
 |-- record_status_indicator: string (nullable = true)
 |-- place_of_receipt: string (nullable = true)
 |-- port_of_destination: string (nullable = true)
 |-- foreign_port_of_destination_qualifier: string (nullable = true)
 |-- foreign_port_of_destination: string (nullable = true)
 |-- conveyance_id_qualifier: string (nullable 

In [266]:
df_container.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- identifier: string (nullable = true)
 |-- container_number: string (nullable = true)
 |-- seal_number_1: string (nullable = true)
 |-- seal_number_2: string (nullable = true)
 |-- equipment_description_code: string (nullable = true)
 |-- container_length: string (nullable = true)
 |-- container_height: string (nullable = true)
 |-- container_width: string (nullable = true)
 |-- container_type: string (nullable = true)
 |-- load_status: string (nullable = true)
 |-- type_of_service: string (nullable = true)

In [267]:
df_shipper.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- identifier: string (nullable = true)
 |-- shipper_party_name: string (nullable = true)
 |-- shipper_party_address_1: string (nullable = true)
 |-- shipper_party_address_2: string (nullable = true)
 |-- shipper_party_address_3: string (nullable = true)
 |-- shipper_party_address_4: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state_province: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- contact_name: string (nullable = true)
 |-- comm_number_qualifier: string (nullable = true)
 |-- comm_number: string (nullable = true)

### Queries and Data Extraction

#### Year-Month Shipment Count

In [268]:
df_shipment = (df_header.groupBy('actual_year_month', 'actual_year', 'actual_month').count()
                        .orderBy('actual_year_month').toPandas())
write_csv = df_shipment.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/year_month_shipment.csv')
df_shipment.head(10)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  actual_year_month  actual_year  actual_month    count
0           2018-01         2018             1  1469569
1           2018-02         2018             2  1493627
2           2018-03         2018             3  1401190
3           2018-04         2018             4  1520733
4           2018-05         2018             5  1862740
5           2018-06         2018             6  1863438
6           2018-07         2018             7  1808743
7           2018-08         2018             8  1737468
8           2018-09         2018             9  1686185
9           2018-10         2018            10  1845571

#### Year Shipment Count

In [269]:
df_shipment = (df_header.groupBy('actual_year').count()
                        .orderBy('actual_year').toPandas())
write_csv = df_shipment.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/year_shipment.csv')
df_shipment.head(10)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   actual_year     count
0         2018  20008181
1         2019  19020357
2         2020  14816356

#### Country Shipment Count

In [270]:
df_country = (df_header.groupBy('vessel_country_code').count()
                        .orderBy(F.desc('count')).toPandas())
df_country["country_name"] = df_country["vessel_country_code"].map(country_codes)
write_csv = df_country.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/country_count.csv')
df_country.head(10)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  vessel_country_code    count      country_name
0                  PA  9120363            Panama
1                  LR  7946666           Liberia
2                  HK  7898450         Hong Kong
3                  SG  4959128         Singapore
4                  GB  3440088    United Kingdom
5                  US  3043480     United States
6                  MT  2813868             Malta
7                  DE  2363417           Germany
8                  MH  2083770  Marshall Islands
9                  DK  1855280           Denmark

#### Top 10 Country Per Year Shipment Count

In [271]:
df_country = (df_header.groupBy('vessel_country_code', 'actual_year').count()
                        .orderBy(F.desc('count')))

window_spec = Window.partitionBy("actual_year").orderBy(df_country["count"].desc())

df_country = df_country.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_country = df_country.filter(df_country.row_num <= N).toPandas()
df_country["country_name"] = df_country["vessel_country_code"].map(country_codes)
write_csv = df_country.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/top10_country_year_trend.csv')
df_country.head(10)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  vessel_country_code  actual_year    count  row_num      country_name
0                  PA         2018  3491737        1            Panama
1                  HK         2018  2860930        2         Hong Kong
2                  LR         2018  2726425        3           Liberia
3                  SG         2018  2209861        4         Singapore
4                  GB         2018  1534067        5    United Kingdom
5                  US         2018  1080124        6     United States
6                  MT         2018   913455        7             Malta
7                  MH         2018   761326        8  Marshall Islands
8                  DE         2018   716469        9           Germany
9                  DK         2018   681675       10           Denmark

In [ ]:
Top 10 Country Per Year Shipment Count

#### Top 10 Country Per Year-Month Shipment Count

In [272]:
df_country = (df_header.groupBy('vessel_country_code', 'actual_year_month', 
                                'actual_year', 'actual_month').count()
                       .orderBy(F.desc('count')))

window_spec = Window.partitionBy("actual_year_month").orderBy(df_country["count"].desc())

df_country = df_country.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_country = df_country.filter(df_country.row_num <= N).toPandas()
df_country["country_name"] = df_country["vessel_country_code"].map(country_codes)
write_csv = df_country.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/top10_country_yearmonth_trend.csv')
df_country.head(10)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  vessel_country_code actual_year_month  ...  row_num      country_name
0                  PA           2018-01  ...        1            Panama
1                  HK           2018-01  ...        2         Hong Kong
2                  LR           2018-01  ...        3           Liberia
3                  SG           2018-01  ...        4         Singapore
4                  GB           2018-01  ...        5    United Kingdom
5                  US           2018-01  ...        6     United States
6                  MT           2018-01  ...        7             Malta
7                  PT           2018-01  ...        8          Portugal
8                  MH           2018-01  ...        9  Marshall Islands
9                  DK           2018-01  ...       10           Denmark

[10 rows x 7 columns]

In [37]:
#### All Country Per Year-Month Shipment Count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
df_country = (df_header.groupBy('vessel_country_code', 'actual_year_month', 
                                'actual_year', 'actual_month').count()
                       .orderBy(F.desc('count')))

window_spec = Window.partitionBy("actual_year_month").orderBy(df_country["count"].desc())

df_country = df_country.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
# N = 10
# df_country = df_country.filter(df_country.row_num <= N).toPandas()

df_country = df_country.toPandas()
df_country["country_name"] = df_country["vessel_country_code"].map(country_codes)
write_csv = df_country.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/all_country_yearmonth_trend.csv')
df_country.head(10)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  vessel_country_code actual_year_month  ...  row_num      country_name
0                  PA           2018-01  ...        1            Panama
1                  HK           2018-01  ...        2         Hong Kong
2                  LR           2018-01  ...        3           Liberia
3                  SG           2018-01  ...        4         Singapore
4                  GB           2018-01  ...        5    United Kingdom
5                  US           2018-01  ...        6     United States
6                  MT           2018-01  ...        7             Malta
7                  PT           2018-01  ...        8          Portugal
8                  MH           2018-01  ...        9  Marshall Islands
9                  DK           2018-01  ...       10           Denmark

[10 rows x 7 columns]

#### Type of Service Count

In [273]:
df_head_cont = (df_header.join(df_container, on='identifier', how='inner'))

df_service = (df_head_cont.where(F.col('type_of_service').isNotNull())
                          .groupBy('type_of_service').count()
                          .orderBy(F.desc('count')).toPandas())

write_csv = df_service.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/type_of_service_count.csv')
df_service.head(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         type_of_service     count
0         House to House  31634356
1         Container Yard  21567303
2           Pier to Pier  15043107
3          Pier to House   3097522
4      Container Station   2218819
5          House to Pier    951812
6             Break Bulk    847594
7      Non Containerized    116907
8       Roll on Roll Off     11396
9  Headload or Devanning      2861

#### Top 10 Type of Service Year Count 

In [275]:
df_service = (df_head_cont.where(F.col('type_of_service').isNotNull())
                          .groupBy('type_of_service', 'actual_year')
                          .count().orderBy(F.desc('count')))

window_spec = Window.partitionBy("actual_year").orderBy(df_service["count"].desc())

df_service = df_service.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_service = df_service.filter(df_service.row_num <= N).toPandas()

write_csv = df_service.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/service_year_trend.csv')
df_service.head(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         type_of_service  actual_year     count  row_num
0         House to House         2018  12183638        1
1         Container Yard         2018   8289124        2
2           Pier to Pier         2018   5184091        3
3          Pier to House         2018   1129427        4
4      Container Station         2018    620207        5
5          House to Pier         2018    363368        6
6             Break Bulk         2018    326929        7
7      Non Containerized         2018     44446        8
8       Roll on Roll Off         2018      4358        9
9  Headload or Devanning         2018      1126       10

#### Top 10 Type of Service Year-Month Count 

In [276]:
df_service = (df_head_cont.where(F.col('type_of_service').isNotNull())
                          .groupBy('type_of_service', 'actual_year_month', 
                                'actual_year', 'actual_month')
                          .count().orderBy(F.desc('count')))

window_spec = Window.partitionBy("actual_year_month").orderBy(df_service["count"].desc())

df_service = df_service.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_service = df_service.filter(df_service.row_num <= N).toPandas()

write_csv = df_service.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/service_yearmonth_trend.csv')
df_service.head(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         type_of_service actual_year_month  ...   count  row_num
0         House to House           2018-01  ...  929014        1
1         Container Yard           2018-01  ...  563851        2
2           Pier to Pier           2018-01  ...  378553        3
3          Pier to House           2018-01  ...   75274        4
4      Container Station           2018-01  ...   42408        5
5             Break Bulk           2018-01  ...   27703        6
6          House to Pier           2018-01  ...   23635        7
7      Non Containerized           2018-01  ...    2891        8
8       Roll on Roll Off           2018-01  ...     211        9
9  Headload or Devanning           2018-01  ...     116       10

[10 rows x 6 columns]

#### Top 10 Type of Service Country Count 

In [277]:
df_ser_count = (df_head_cont.where(F.col('type_of_service').isNotNull())
                          .groupBy('type_of_service', 'vessel_country_code')
                          .count().orderBy(F.desc('count')))

window_spec = Window.partitionBy("type_of_service").orderBy(df_ser_count["count"].desc())

df_ser_count = df_ser_count.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_ser_count = df_ser_count.filter(df_ser_count.row_num <= N).toPandas()
df_ser_count["country_name"] = df_ser_count["vessel_country_code"].map(country_codes)
write_csv = df_ser_count.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/top10_country_count_per_service.csv')
df_ser_count.head(20)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      type_of_service vessel_country_code   count  row_num      country_name
0          Break Bulk                  PA  144811        1            Panama
1          Break Bulk                  HK  126783        2         Hong Kong
2          Break Bulk                  LR  110037        3           Liberia
3          Break Bulk                  SG   75425        4         Singapore
4          Break Bulk                  GB   61357        5    United Kingdom
5          Break Bulk                  DE   53964        6           Germany
6          Break Bulk                  US   42294        7     United States
7          Break Bulk                  MT   38383        8             Malta
8          Break Bulk                  PT   29115        9          Portugal
9          Break Bulk                  MH   28531       10  Marshall Islands
10  Container Station                  LR  365572        1           Liberia
11  Container Station                  HK  278816        2         Hong Kong

#### Top 10 Type of Service Country Year Count 

In [278]:
df_ser_count = (df_head_cont.where(F.col('type_of_service').isNotNull())
                          .groupBy('type_of_service', 'actual_year', 'vessel_country_code')
                          .count().orderBy(F.desc('count')))

window_spec = Window.partitionBy(["type_of_service","actual_year"]).orderBy(df_ser_count["count"].desc())

df_ser_count = df_ser_count.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_ser_count = df_ser_count.filter(df_ser_count.row_num <= N).toPandas()
df_ser_count["country_name"] = df_ser_count["vessel_country_code"].map(country_codes)
write_csv = df_ser_count.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/top10_trend_year_service_country.csv')
df_ser_count.head(20)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   type_of_service  actual_year  ... row_num      country_name
0       Break Bulk         2018  ...       1            Panama
1       Break Bulk         2018  ...       2         Hong Kong
2       Break Bulk         2018  ...       3           Liberia
3       Break Bulk         2018  ...       4    United Kingdom
4       Break Bulk         2018  ...       5         Singapore
5       Break Bulk         2018  ...       6             Malta
6       Break Bulk         2018  ...       7          Portugal
7       Break Bulk         2018  ...       8  Marshall Islands
8       Break Bulk         2018  ...       9           Germany
9       Break Bulk         2018  ...      10     United States
10      Break Bulk         2019  ...       1            Panama
11      Break Bulk         2019  ...       2           Liberia
12      Break Bulk         2019  ...       3         Hong Kong
13      Break Bulk         2019  ...       4         Singapore
14      Break Bulk         2019  ...       5           

#### Top 10 Type of Service Country Year-Month Count 

In [279]:
df_ser_count = (df_head_cont.where(F.col('type_of_service').isNotNull())
                          .groupBy('type_of_service', 'actual_year', 'actual_year', 
                                   'actual_year_month', 'vessel_country_code')
                          .count().orderBy(F.desc('count')))

window_spec = Window.partitionBy(["type_of_service","actual_year_month"]).orderBy(df_ser_count["count"].desc())

df_ser_count = df_ser_count.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_ser_count = df_ser_count.filter(df_ser_count.row_num <= N).toPandas()
df_ser_count["country_name"] = df_ser_count["vessel_country_code"].map(country_codes)

write_csv = df_ser_count.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/top10_trend_yearmonth_service_country.csv')
df_ser_count.head(20)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   type_of_service  actual_year  actual_year  ... count row_num      country_name
0       Break Bulk         2018         2018  ...  4924       1         Hong Kong
1       Break Bulk         2018         2018  ...  4200       2           Liberia
2       Break Bulk         2018         2018  ...  4010       3            Panama
3       Break Bulk         2018         2018  ...  3819       4    United Kingdom
4       Break Bulk         2018         2018  ...  2469       5         Singapore
5       Break Bulk         2018         2018  ...  1472       6  Marshall Islands
6       Break Bulk         2018         2018  ...  1161       7     United States
7       Break Bulk         2018         2018  ...  1096       8             Malta
8       Break Bulk         2018         2018  ...   922       9          Portugal
9       Break Bulk         2018         2018  ...   545      10           Denmark
10      Break Bulk         2018         2018  ...  4640       1            Panama
11      Break Bu

In [54]:
df_head_cont = None
df_head_ship = (df_header.join(df_shipper, on='identifier', how='inner'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Top 1000 Shipper Count

In [282]:
df_shippers = (df_head_ship.groupby('shipper_party_name', )
                           .agg(F.count("*").alias('count'), 
                                F.first('shipper_party_address_1').alias('address1'),
                                F.first('shipper_party_address_2').alias('address2'),
                                F.first('shipper_party_address_3').alias('address3'),
                                F.first('shipper_party_address_4').alias('address4'),
                                F.first('country_code').alias('country_code')
                                )
                           .orderBy(F.desc('count'))
                           .limit(1000).toPandas())

write_csv = df_shippers.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/top1000_shipper_country.csv')
df_shippers.head(20)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     shipper_party_name  ...  country_code
0    ORIENT EXPRESS CONTAINER CO., LTD.  ...          None
1                      WANG ZHANG JIANG  ...          None
2          HONOUR LANE SHIPPING LIMITED  ...          None
3   BEIJING KANG JIE KONG INTERNATIONAL  ...          None
4                HECNY SHIPPING LIMITED  ...            21
5       DE WELL CONTAINER SHIPPING INC.  ...          None
6        TOPOCEAN CONSOLIDATION SERVICE  ...            26
7       FOREVER LOGISTICS SHENZHEN CO L  ...          None
8             PANTOS LOGISTICS CO.,LTD.  ...          None
9           FEDEX INTERNATIONAL FREIGHT  ...          None
10  UNION DE BANANEROS ECUATORIANOS S.A  ...          None
11            HONOUR LANE SHIPPING LTD.  ...          None
12  SCHAEFFLER TECHNOLOGIES AG & CO. KG  ...          None
13        INTERGLOBO NORTH AMERICA INC.  ...          None
14         SAMSUNG ELECTRONICS CO., LTD  ...          None
15   U.S.UNITED LOGISTICS (NINGBO) INC.  ...          No

In [283]:
#df_head_ship = None


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [284]:
df_delay = df_header.select(F.mean('delay')).toPandas()

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/mean_delay.csv')
df_delay.head(20)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   avg(delay)
0    1.889987

#### On Schedule Delay Early Count

In [26]:
row_cnt = df_header.count()
df_delay = (df_header.groupBy('is_delayed').count()
                     .select('is_delayed',  'count', 
                            (F.col('count')/F.lit(row_cnt) 
                            * 100).alias('%') ).toPandas())

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/delay_distri.csv')
df_delay.head(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    is_delayed     count          %
0  on schedule  19096748  35.466219
1      delayed  30301635  56.275782
2        early   4446511   8.257999

#### On Schedule Delay Early Year Count

In [33]:
row_cnt = df_header.count()

window_spec = Window.partitionBy("estimated_year")

df_delay = (df_header.withColumn("row_count", count("*").over(window_spec))
                     .groupBy('is_delayed', 'estimated_year')
                     .agg(F.count('*').alias('count'), F.first('row_count').alias('row_count'))
                     .select('is_delayed', 'estimated_year', 'count',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'),
                            (F.col('count')/F.col('row_count')*100).alias('% Year-Total')).toPandas())

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/delay_distri_yearly.csv')
df_delay.head(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    is_delayed  estimated_year     count    % Total  % Year-Total
0        early            2018   1549380   2.877487      7.668285
1      delayed            2018  12122935  22.514549     59.999560
2  on schedule            2018   6532725  12.132487     32.332156
3        early            2020   1439603   2.673611      9.792823
4      delayed            2020   8134259  15.106834     55.332867
5  on schedule            2020   5126730   9.521293     34.874310
6        early            2019   1457528   2.706901      7.695801
7  on schedule            2019   7437293  13.812439     39.269181
8      delayed            2019  10044441  18.654398     53.035018

#### Delay Year-Month Count

In [44]:
row_cnt = df_header.where(F.col('is_delayed') == 'delayed').count()
window_spec = Window.partitionBy("estimated_year")
df_delay = (df_header.where(F.col('is_delayed') == 'delayed')
                     .withColumn("row_count", count("*").over(window_spec))
                     .groupBy('is_delayed', 'estimated_year_month')
                     .agg(F.count('*').alias('count'), F.first('row_count').alias('row_count'))
                     .select('is_delayed', 'estimated_year_month', 'count',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'),
                            (F.col('count')/F.col('row_count')*100).alias('% Year-Total')).toPandas())

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/delayed_only_distri_year_month.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   is_delayed estimated_year_month    count   % Total  % Year-Total
0     delayed              2018-04   883468  2.915579      7.287575
1     delayed              2018-08   922820  3.045446      7.612183
2     delayed              2018-11  1054841  3.481136      8.701201
3     delayed              2018-09  1076607  3.552967      8.880745
4     delayed              2018-01  1070203  3.531833      8.827920
5     delayed              2018-10  1207982  3.986524      9.964435
6     delayed              2018-05   961981  3.174683      7.935215
7     delayed              2018-06   855179  2.822221      7.054224
8     delayed              2018-03   906349  2.991089      7.476317
9     delayed              2018-07   906969  2.993136      7.481431
10    delayed              2018-02  1001184  3.304059      8.258594
11    delayed              2018-12  1275352  4.208855     10.520159
12    delayed              2019-10   766682  2.530167      7.632899
13    delayed              2019-08   755466  2.4

#### Delay Early Year-Month Count

In [45]:
row_cnt = df_header.count()

window_spec = Window.partitionBy("estimated_year_month")

df_delay = (df_header.withColumn("row_count", count("*").over(window_spec))
                     .groupBy('is_delayed', 'estimated_year_month')
                     .agg(F.count('*').alias('count'), F.first('row_count').alias('row_count'))
                     .select('is_delayed', 'estimated_year_month', 'count',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'),
                            (F.col('count')/F.col('row_count')*100).alias('% Year-Month-Total')).toPandas())

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/delay_early_distri_year_month.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     is_delayed estimated_year_month    count   % Total  % Year-Month-Total
0         early              2020-05   209505  0.389090           13.578862
1       delayed              2020-05   606877  1.127084           39.334140
2   on schedule              2020-05   726494  1.349235           47.086999
3         early              2020-11       57  0.000106          100.000000
4         early              2020-07   175758  0.326415            9.249730
5       delayed              2020-07  1017451  1.889596           53.546051
6   on schedule              2020-07   706933  1.312906           37.204219
7       delayed              2019-08   755466  1.403041           44.525609
8   on schedule              2019-08   807140  1.499009           47.571168
9         early              2019-08   134094  0.249038            7.903224
10  on schedule              2019-09   669182  1.242796           42.031537
11      delayed              2019-09   809683  1.503732           50.856450
12        ea

#### Mean Delay Country Count

In [288]:
row_cnt = df_header.count()

df_delay = (df_header.groupBy('vessel_country_code')
                     .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                     .select('vessel_country_code', 'count', 'mean_delay',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'))
                     .where(F.col('count') >= 50)
                     .orderBy(F.desc('mean_delay')).toPandas())
df_delay["country_name"] = df_delay["vessel_country_code"].map(country_codes)
write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/mean_country_delay_and_early.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   vessel_country_code   count  mean_delay   % Total          country_name
0                   GQ      74    9.216216  0.000137     Equatorial Guinea
1                   AD   41508    8.951720  0.077088               Andorra
2                   CV     774    8.757106  0.001437            Cape Verde
3                   GW      63    8.206349  0.000117         Guinea-Bissau
4                   ET      59    8.101695  0.000110              Ethiopia
5                   BO      98    7.581633  0.000182               Bolivia
6                   QA    1785    6.144538  0.003315                 Qatar
7                   TR   43365    6.018102  0.080537                Turkey
8                   BG      80    5.287500  0.000149              Bulgaria
9                   AQ      55    5.200000  0.000102                   NaN
10                  KW    4190    5.088305  0.007782                Kuwait
11                  AE   25029    4.598426  0.046484  United Arab Emirates
12                  AW   

#### Sorted by Mean Delay Country Count

In [289]:
df_delay.sort_values(by='mean_delay').head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    vessel_country_code   count  mean_delay   % Total            country_name
163                  NC      65  -20.846154  0.000121           New Caledonia
162                  NZ     834  -18.488010  0.001549             New Zealand
161                  SA   10996   -1.718807  0.020422            Saudi Arabia
160                  PF     212   -1.716981  0.000394        French Polynesia
159                  MZ     494   -1.149798  0.000917              Mozambique
158                  SK     146   -1.047945  0.000271                Slovakia
157                  MN     107   -0.467290  0.000199                Mongolia
156                  CL    1912    0.008891  0.003551                   Chile
155                  KG      99    0.020202  0.000184              Kyrgyzstan
154                  FO     220    0.063636  0.000409           Faroe Islands
153                  EE      88    0.102273  0.000163                 Estonia
152                  GU      51    0.117647  0.000095           

#### Sorted by Mean Delay Country To Check Early Shippers 

In [36]:
row_cnt = df_header.where(F.col('is_delayed') == 'delayed').count()

df_delay = (df_header.where(F.col('is_delayed') == 'delayed')
                     .groupBy('vessel_country_code')
                     .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                     .select('vessel_country_code', 'count', 'mean_delay',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'))
                     .orderBy(F.desc('count')).toPandas())
df_delay["country_name"] = df_delay["vessel_country_code"].map(country_codes)
write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/delayed_country_count.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   vessel_country_code    count  mean_delay    % Total          country_name
0                   PA  5817596    3.781203  19.198951                Panama
1                   HK  5204204    4.204467  17.174664             Hong Kong
2                   LR  4349193    3.958279  14.352998               Liberia
3                   SG  2435831    3.289145   8.038612             Singapore
4                   GB  1612495    5.619168   5.321479        United Kingdom
5                   MT  1597931    4.406596   5.273415                 Malta
6                   DE  1475025    5.029873   4.867807               Germany
7                   MH  1301019    4.084814   4.293560      Marshall Islands
8                   US  1286078    3.159671   4.244253         United States
9                   DK   894329    3.122431   2.951422               Denmark
10                  PT   863843    3.661480   2.850813              Portugal
11                  JP   629927    4.164778   2.078855                 Japan

#### Sorted by Count Early Country Count

In [291]:
row_cnt = df_header.where(F.col('is_delayed') != 'delayed').count()

window_spec = Window.partitionBy("estimated_year")

df_delay = (df_header.where(F.col('is_delayed') != 'delayed')
                     .groupBy('vessel_country_code')
                     .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_early'))
                     .select('vessel_country_code', 'count', 'mean_early',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'))
                     .orderBy(F.desc('count')).toPandas())

df_delay["country_name"] = df_delay["vessel_country_code"].map(country_codes)

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/early_country_count.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   vessel_country_code    count  ...    % Total                      country_name
0                   LR  3597473  ...  15.280268                           Liberia
1                   PA  3302767  ...  14.028504                            Panama
2                   HK  2694246  ...  11.443811                         Hong Kong
3                   SG  2523297  ...  10.717705                         Singapore
4                   GB  1827593  ...   7.762702                    United Kingdom
5                   US  1757402  ...   7.464566                     United States
6                   MT  1215937  ...   5.164693                             Malta
7                   DK   960951  ...   4.081640                           Denmark
8                   DE   888392  ...   3.773445                           Germany
9                   MH   782751  ...   3.324735                  Marshall Islands
10                  PT   775575  ...   3.294255                          Portugal
11              

#### Top 10 S Mean Delay Country Sorted Count

In [292]:
row_cnt = df_header.where(F.col('is_delayed') == 'delayed').count()

df_delay = (df_header.where(F.col('is_delayed') == 'delayed')
                     .groupBy('vessel_country_code', 'actual_year')
                     .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                     .orderBy(F.desc('count'))
                     .where(F.col('count') >= 50))

window_spec = Window.partitionBy(["actual_year"]).orderBy(df_delay["count"].desc())

df_delay = df_delay.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_delay = df_delay.filter(df_delay.row_num <= N).toPandas()
df_delay["country_name"] = df_delay["vessel_country_code"].map(country_codes)

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/delayed_country_count_year.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   vessel_country_code  actual_year  ...  row_num      country_name
0                   PA         2018  ...        1            Panama
1                   HK         2018  ...        2         Hong Kong
2                   LR         2018  ...        3           Liberia
3                   SG         2018  ...        4         Singapore
4                   GB         2018  ...        5    United Kingdom
5                   MT         2018  ...        6             Malta
6                   MH         2018  ...        7  Marshall Islands
7                   US         2018  ...        8     United States
8                   DE         2018  ...        9           Germany
9                   PT         2018  ...       10          Portugal
10                  PA         2019  ...        1            Panama
11                  HK         2019  ...        2         Hong Kong
12                  LR         2019  ...        3           Liberia
13                  SG         2019  ...        

#### Top 10 Mean Country Delayed Count

In [293]:
row_cnt = df_header.where(F.col('is_delayed') == 'delayed').count()

df_delay = (df_header.where(F.col('is_delayed') == 'delayed')
                     .groupBy('vessel_country_code', 'actual_year')
                     .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                     .orderBy(F.desc('count'))
                     .where(F.col('count') >= 50))

window_spec = Window.partitionBy(["actual_year"]).orderBy(df_delay["mean_delay"].desc())

df_delay = df_delay.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_delay = df_delay.filter(df_delay.row_num <= N).toPandas()
df_delay["country_name"] = df_delay["vessel_country_code"].map(country_codes)

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/delayed_country_mean_desc_year.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   vessel_country_code  actual_year  ...  row_num           country_name
0                   SA         2018  ...        1           Saudi Arabia
1                   TR         2018  ...        2                 Turkey
2                   VN         2018  ...        3                Vietnam
3                   QA         2018  ...        4                  Qatar
4                   PL         2018  ...        5                 Poland
5                   BR         2018  ...        6                 Brazil
6                   GI         2018  ...        7              Gibraltar
7                   IN         2018  ...        8                  India
8                   CR         2018  ...        9             Costa Rica
9                   CL         2018  ...       10                  Chile
10                  AD         2019  ...        1                Andorra
11                  GQ         2019  ...        2      Equatorial Guinea
12                  CV         2019  ...        3  

#### Top 10 Mean Country Early Sorted Count

In [ ]:
row_cnt = df_header.where(F.col('is_delayed') != 'delayed').count()

df_delay = (df_header.where(F.col('is_delayed') != 'delayed')
                     .groupBy('vessel_country_code', 'actual_year')
                     .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                     .orderBy(F.desc('count'))
                     .where(F.col('count') >= 50))

window_spec = Window.partitionBy(["actual_year"]).orderBy(df_delay["count"].desc())

df_delay = df_delay.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_delay = df_delay.filter(df_delay.row_num <= N).toPandas()
df_delay["country_name"] = df_delay["vessel_country_code"].map(country_codes)

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/early_country_count_year.csv')
df_delay.head(30)

#### Top 10 Mean Country Early Year Count

In [295]:
row_cnt = df_header.where(F.col('is_delayed') != 'delayed').count()

df_delay = (df_header.where(F.col('is_delayed') != 'delayed')
                     .groupBy('vessel_country_code', 'actual_year')
                     .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                     .orderBy(F.desc('count'))
                     .where(F.col('count') >= 50))

window_spec = Window.partitionBy(["actual_year"]).orderBy(df_delay["mean_delay"])

df_delay = df_delay.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_delay = df_delay.filter(df_delay.row_num <= N).toPandas()
df_delay["country_name"] = df_delay["vessel_country_code"].map(country_codes)

write_csv = df_delay.to_csv(index=False)

s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/early_country_mean_desc_year.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   vessel_country_code  actual_year  count  mean_delay  row_num    country_name
0                   MZ         2018     51  -17.078431        1      Mozambique
1                   KY         2018   1183   -6.044801        2  Cayman Islands
2                   GE         2018    548   -5.521898        3         Georgia
3                   ML         2018     70   -5.228571        4            Mali
4                   NZ         2018    231   -4.956710        5     New Zealand
5                   CL         2018     79   -4.215190        6           Chile
6                   BH         2018    208   -4.081731        7         Bahrain
7                   ID         2018    133   -3.233083        8       Indonesia
8                   AF         2018    600   -2.903333        9     Afghanistan
9                   SA         2018    203   -2.871921       10    Saudi Arabia
10                  NZ         2019    565  -25.522124        1     New Zealand
11                  VN         2019    1

In [307]:
row_cnt = df_head_ship.count()

df_delay = (df_head_ship.groupBy('shipper_party_name')
                        .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                        .select('shipper_party_name', 'count', 'mean_delay',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'))
                        .where(F.col('count') >= 50)
                        .orderBy(F.desc('mean_delay')).toPandas())

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/mean_order_mean_shipper_delay_early.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     shipper_party_name  count  mean_delay   % Total
0    KUNSHAN LUXSHARE RF TECHNOLOGY LTD     62  126.806452  0.000155
1               GALLOTTI AND RADICE SRL     70  102.571429  0.000175
2         DOLPHIN APPAREL INTERNATIONAL     55  101.090909  0.000138
3            KEIN HING INDUSTRY SDN BHD    270   99.874074  0.000676
4                      HEINZ ITALIA SPA     54   89.907407  0.000135
5               THE FRESH OLIVE COMPANY     62   87.516129  0.000155
6                FRACCARO SPUMADORO SPA     58   86.034483  0.000145
7                BURGDORF GMBH & CO. KG     56   84.178571  0.000140
8     HUBEI GRAND LIFE SCIENCE TECHNOLO     78   83.679487  0.000195
9                  SANTUCCI ARMANDO SRL     70   83.357143  0.000175
10                 LAPP GMBH KABELWERKE     77   81.675325  0.000193
11                              H07 SRL     62   79.870968  0.000155
12    OFFICINA DESIGN DI SIALINO ILARIA     58   79.827586  0.000145
13                 BKF AEROSPACE C

#### Mean Shipper Delay Early Count

In [308]:
df_delay = (df_head_ship.groupBy('shipper_party_name')
                        .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                        .select('shipper_party_name', 'count', 'mean_delay',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'))
                        .where(F.col('count') >= 50)
                        .orderBy(F.desc('count')).toPandas())

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/mean_order_count_shipper_delay_early.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     shipper_party_name   count  mean_delay   % Total
0    ORIENT EXPRESS CONTAINER CO., LTD.  162531    1.798900  0.406746
1                      WANG ZHANG JIANG  150065    0.192863  0.375549
2          HONOUR LANE SHIPPING LIMITED  126809    1.524884  0.317349
3   BEIJING KANG JIE KONG INTERNATIONAL   97340    1.653318  0.243601
4                HECNY SHIPPING LIMITED   83004    1.505590  0.207724
5       DE WELL CONTAINER SHIPPING INC.   81762    1.514738  0.204615
6        TOPOCEAN CONSOLIDATION SERVICE   66164    1.589112  0.165580
7       FOREVER LOGISTICS SHENZHEN CO L   55093   -0.500100  0.137874
8             PANTOS LOGISTICS CO.,LTD.   55046    1.609835  0.137757
9           FEDEX INTERNATIONAL FREIGHT   53171    2.360309  0.133064
10  UNION DE BANANEROS ECUATORIANOS S.A   46503    0.026256  0.116377
11            HONOUR LANE SHIPPING LTD.   45873    1.753515  0.114801
12  SCHAEFFLER TECHNOLOGIES AG & CO. KG   45759    0.616076  0.114515
13        INTERGLOBO

In [309]:
row_cnt = df_head_ship.where(F.col('is_delayed') == 'delayed').count()

df_delay = (df_head_ship.where(F.col('is_delayed') == 'delayed')
                        .groupBy('shipper_party_name')
                        .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                        .select('shipper_party_name', 'count', 'mean_delay',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'))
                        .where(F.col('count') >= 50)
                        .orderBy(F.desc('mean_delay')).toPandas())

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/mean_order_mean_shipper_delay.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     shipper_party_name  count  mean_delay   % Total
0            KEIN HING INDUSTRY SDN BHD    188  144.515957  0.000840
1     HUBEI GRAND LIFE SCIENCE TECHNOLO     51  128.156863  0.000228
2           WENZHOU HOS GARMENT CO.,LTD    107  128.074766  0.000478
3    GE HEALTHCARE AUSTRIA GMBH & CO OG     65  118.276923  0.000290
4   ANHUI HELI INDUSTRIAL VEHICLE IMP &     53  107.018868  0.000237
5         DOLPHIN APPAREL INTERNATIONAL     52  106.923077  0.000232
6                          LAITA EXPORT     60  105.800000  0.000268
7    APLICACIONES MECANICAS DEL CAUCHO,     60  101.533333  0.000268
8                            DEFI GROUP     62  100.080645  0.000277
9                  LAPP GMBH KABELWERKE     63   99.984127  0.000282
10               FRACCARO SPUMADORO SPA     50   99.800000  0.000223
11                              CASSESE     78   95.679487  0.000349
12                 SANTUCCI ARMANDO SRL     62   94.145161  0.000277
13         ORKLA CONFECTIONERY & S

#### Mean Shipper Delay Count

In [310]:
row_cnt = df_head_ship.where(F.col('is_delayed') == 'delayed').count()

df_delay = (df_head_ship.where(F.col('is_delayed') == 'delayed')
                        .groupBy('shipper_party_name')
                        .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                        .select('shipper_party_name', 'count', 'mean_delay',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'))
                        .where(F.col('count') >= 50)
                        .orderBy(F.desc('count')).toPandas())

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/mean_order_count_shipper_delay.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     shipper_party_name   count  mean_delay   % Total
0    ORIENT EXPRESS CONTAINER CO., LTD.  107912    2.783907  0.482252
1          HONOUR LANE SHIPPING LIMITED   72979    2.767920  0.326139
2   BEIJING KANG JIE KONG INTERNATIONAL   59164    2.874079  0.264400
3                HECNY SHIPPING LIMITED   48700    2.757084  0.217637
4       DE WELL CONTAINER SHIPPING INC.   47692    2.747044  0.213133
5        TOPOCEAN CONSOLIDATION SERVICE   35711    3.091344  0.159590
6             PANTOS LOGISTICS CO.,LTD.   34535    2.647112  0.154335
7           FEDEX INTERNATIONAL FREIGHT   32685    3.903778  0.146067
8   BEIJING KANG-JIE-KONG INTERNATIONAL   28097    3.228494  0.125564
9             HONOUR LANE SHIPPING LTD.   27539    3.044628  0.123070
10           PUDONG PRIME INTERNATIONAL   25359    2.944398  0.113328
11        INTERGLOBO NORTH AMERICA INC.   24846    2.941158  0.111035
12         SAMSUNG ELECTRONICS CO., LTD   24707    2.791921  0.110414
13  ORIENT STAR TRAN

In [311]:
row_cnt = df_head_ship.where(F.col('is_delayed') != 'delayed').count()

df_delay = (df_head_ship.where(F.col('is_delayed') != 'delayed')
                        .groupBy('shipper_party_name')
                        .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                        .select('shipper_party_name', 'count', 'mean_delay',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'))
                        .where(F.col('count') >= 50)
                        .orderBy(F.col('mean_delay')).toPandas())

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/mean_order_mean_shipper_early.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     shipper_party_name  count  mean_delay   % Total
0                          DAALIMPEX BV    137  -78.905109  0.000779
1                         LIKEMEAT GMBH     83  -77.759036  0.000472
2          PERFETTI VAN MELLE GIDA SAN.     52  -72.211538  0.000296
3   FOSHAN NANHAI RONGJIA IMPORT & EXPO     67  -71.537313  0.000381
4    KUNSHAN RED WOOD ENTERPRISE CO.,LT     51  -66.960784  0.000290
5              GEMORE TECHNOLOGY CO LTD     52  -59.057692  0.000296
6         AGRIGUM INTERNATIONAL LIMITED     59  -58.355932  0.000336
7                     GRADCO JAPAN LTD.     51  -57.392157  0.000290
8                                 SOMFY    160  -56.975000  0.000910
9                   ACCSYS TECHNOLOGIES     69  -51.318841  0.000392
10                        JIHOSTROJ A S    162  -51.055556  0.000921
11                GRACEFUL TEAM LIMITED     62  -50.564516  0.000353
12    GUANGDONG DEHE TECHNOLOGY CO.,LTD     82  -48.804878  0.000466
13       PT. DONG YANG ILLUST INDO

#### Mean Shipper Delay Early Sorted By Count Min 50 Shipments

In [41]:
row_cnt = df_head_ship.where(F.col('is_delayed') != 'delayed').count()

df_delay = (df_head_ship.where(F.col('is_delayed') != 'delayed')
                        .groupBy('shipper_party_name')
                        .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                        .select('shipper_party_name', 'count', 'mean_delay',
                            (F.col('count')/F.lit(row_cnt)*100).alias('% Total'))
                        .where(F.col('count') >= 50)
                        .orderBy(F.desc('count')).toPandas())

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/mean_order_count_shipper_early.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     shipper_party_name   count  mean_delay   % Total
0                      WANG ZHANG JIANG  126208   -0.559711  0.717817
1    ORIENT EXPRESS CONTAINER CO., LTD.   54619   -0.147202  0.310649
2          HONOUR LANE SHIPPING LIMITED   53830   -0.160338  0.306162
3       FOREVER LOGISTICS SHENZHEN CO L   50549   -0.724841  0.287501
4   UNION DE BANANEROS ECUATORIANOS S.A   42539   -0.082842  0.241944
5     SHENZHEN GUANGCHI LOGISTICS CO LT   41656   -0.531256  0.236921
6   BEIJING KANG JIE KONG INTERNATIONAL   38176   -0.238579  0.217129
7                HECNY SHIPPING LIMITED   34304   -0.271105  0.195106
8       DE WELL CONTAINER SHIPPING INC.   34070   -0.210273  0.193776
9   SCHAEFFLER TECHNOLOGIES AG & CO. KG   31269   -3.383383  0.177845
10       TOPOCEAN CONSOLIDATION SERVICE   30453   -0.172495  0.173204
11         ROYAL CARIBBEAN CRUISES LTD.   28333   -0.043871  0.161146
12  SHENZHEN GLOBAL LEADER SUPPLY CHAIN   26010   -1.597732  0.147934
13   U.S.UNITED LOGI

### Mean Delayed Shipper Min 50 Shipments

In [314]:
row_cnt = df_head_ship.where(F.col('is_delayed') == 'delayed').count()

df_delay = (df_head_ship.where(F.col('is_delayed') == 'delayed')
                        .groupBy('shipper_party_name', 'actual_year')
                        .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                        .orderBy(F.desc('count'))
                        .where(F.col('count') >= 50))

window_spec = Window.partitionBy(["actual_year"]).orderBy(df_delay["mean_delay"].desc())

df_delay = df_delay.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_delay = df_delay.filter(df_delay.row_num <= N).toPandas()

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/delayed_country_mean_desc_year.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     shipper_party_name  actual_year  ...  mean_delay  row_num
0        SCHAEFFLER TECH GMBH AND CO KG         2018  ...  125.103448        1
1   BECTON DICKINSON DE MEXICO SA DE CV         2018  ...   99.892323        2
2                        EICHHOLTZ B.V.         2018  ...   98.106061        3
3                             XBK KABEL         2018  ...   92.096154        4
4             GE JENBACHER GMBH & CO OG         2018  ...   82.108696        5
5                        U.I. LAPP GMBH         2018  ...   76.944444        6
6                           NEDMAG B.V.         2018  ...   72.776471        7
7                     DR. OTTO SUWELACK         2018  ...   68.972973        8
8              HAHNEMUEHLE FINEART GMBH         2018  ...   66.147826        9
9                   BRAVILOR BONAMAT BV         2018  ...   64.444444       10
10   NUOVA INDUSTRIA BISCOTTI CRICH SPA         2020  ...  156.607843        1
11                         BRENNTAG SPA         2020

### Mean Eearly Shipper Sort Count Min 50 Shipments

In [315]:
row_cnt = df_head_ship.where(F.col('is_delayed') != 'delayed').count()

df_delay = (df_head_ship.where(F.col('is_delayed') != 'delayed')
                        .groupBy('shipper_party_name', 'actual_year')
                        .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                        .orderBy(F.desc('count'))
                        .where(F.col('count') >= 50))

window_spec = Window.partitionBy(["actual_year"]).orderBy(df_delay["count"].desc())

df_delay = df_delay.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_delay = df_delay.filter(df_delay.row_num <= N).toPandas()

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/early_shipper_count_year.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     shipper_party_name  actual_year  ...  mean_delay  row_num
0          HONOUR LANE SHIPPING LIMITED         2018  ...   -0.186350        1
1    ORIENT EXPRESS CONTAINER CO., LTD.         2018  ...   -0.205433        2
2   UNION DE BANANEROS ECUATORIANOS S.A         2018  ...   -0.107735        3
3       DE WELL CONTAINER SHIPPING INC.         2018  ...   -0.156762        4
4                HECNY SHIPPING LIMITED         2018  ...   -0.223856        5
5        TOPOCEAN CONSOLIDATION SERVICE         2018  ...   -0.242849        6
6   BEIJING KANG JIE KONG INTERNATIONAL         2018  ...   -0.293714        7
7          ROYAL CARIBBEAN CRUISES LTD.         2018  ...   -0.023858        8
8                  CARNIVAL CRUISE LINE         2018  ...   -0.028150        9
9                     NCL (BAHAMAS) LTD         2018  ...   -0.001220       10
10                     WANG ZHANG JIANG         2020  ...   -0.559711        1
11      FOREVER LOGISTICS SHENZHEN CO L         2020

#### Top 10 Mean Shipper Early Count Min 50 Shipments

In [316]:
row_cnt = df_head_ship.where(F.col('is_delayed') != 'delayed').count()

df_delay = (df_head_ship.where(F.col('is_delayed') != 'delayed')
                        .groupBy('shipper_party_name', 'actual_year')
                        .agg(F.count('*').alias('count'), F.mean('delay').alias('mean_delay'))
                        .orderBy(F.desc('count'))
                        .where(F.col('count') >= 50))

window_spec = Window.partitionBy(["actual_year"]).orderBy(df_delay["mean_delay"])

df_delay = df_delay.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_delay = df_delay.filter(df_delay.row_num <= N).toPandas()

write_csv = df_delay.to_csv(index=False)
s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/early_shipper_mean_desc_year.csv')
df_delay.head(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                     shipper_party_name  actual_year  ...  mean_delay  row_num
0                          DAALIMPEX BV         2018  ... -100.274725        1
1                         LIKEMEAT GMBH         2018  ...  -89.166667        2
2          CHAMAN LAL SETIA EXPORTS LTD         2018  ...  -68.535211        3
3                  BYK NETHERLANDS B.V.         2018  ...  -63.568345        4
4   ZHEJIANG XINGXING REFRIGERATION CO.         2018  ...  -63.055556        5
5              POLYPLEX CORPORATION LTD         2018  ...  -57.841667        6
6                         SIGMA-ALDRICH         2018  ...  -53.103448        7
7               ZIEHL-ABEGG DANMARK APS         2018  ...  -48.538462        8
8          TRUMPF MASCH.AUSTR.GMBH&COKG         2018  ...  -47.376344        9
9                        BOBCAT COMPANY         2018  ...  -42.412698       10
10    GUANGDONG DEHE TECHNOLOGY CO.,LTD         2020  ...  -49.370370        1
11                       BOBCAT COMPANY         2020